In [7]:
import requests
import base64
import datetime
from urllib.parse import urlencode

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time
import re
import json
from datetime impport datetime, time, date

import datetime
import time

from requests.exceptions import SSLError

from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'sklearn'

In [2]:
client_id = 'f78a4f4cfe9c40ea8fe346b0576e98ea'
client_secret = 'c26db2d4c1fb42d79dc99945b2360ab4'

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        '''
        returns a base 64 encoded string
        '''
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("Must set client_id and client_secret")
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_header(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            'Authorization' : f'Basic {client_creds_b64}' # Basic <base64 encoded client_id:client_secret>
        }
    
    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'
        }
    
    def auth(self):
        token_data = self.get_token_data()
        token_headers = self.get_token_header()
        token_url = self.token_url

        
        r = requests.post(token_url, data = token_data, headers = token_headers)
        
        
        if r.status_code not in range (200, 299):
            return False
        token_response_data = r.json()
        now = datetime.datetime.now()
        access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        self.access_token = access_token
        return True
        
    def searchTrack(self, item):
        access_token = self.access_token
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({'q': item, 'type':'track'})

        lookup_url = f'{endpoint}?{data}'
        print(lookup_url)

        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        return r.json()
    
    def searchAlbum(self, item):
        access_token = self.access_token
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({'q': item, 'type':'album'})

        lookup_url = f'{endpoint}?{data}'
        print(lookup_url)

        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        return r.json()
    
    def searchArtist(self, item):
        access_token = self.access_token
        
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({'q': item, 'type':'artist'})

        lookup_url = f'{endpoint}?{data}'
        print(lookup_url)

        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        return r.json()
    
    
    
    

In [4]:
client = SpotifyAPI(client_id, client_secret)
client.auth()

True

In [5]:
r = client.searchArtist('the eden project')
r['artists']['items'][0]['genres']

https://api.spotify.com/v1/search?q=the+eden+project&type=artist
200


['indie poptimism', 'modern rock', 'pop', 'pop rap']

In [6]:
# Set Up Streaming History
streamingHistory = []

for f in os.listdir('data/data_2019'):
    if f.startswith('StreamingHistory'):
        streamingHistory.append(pd.read_json(os.path.join('data/data_2019', f)))
    
        
rawHist = pd.concat(streamingHistory).reset_index(drop=True)

In [7]:
rawHist

,endTime,artistName,trackName,msPlayed
0,2018-11-14 00:01,Imagine Dragons,Stuck,190853
1,2018-11-14 00:04,Imagine Dragons,Love,166360
2,2018-11-14 00:08,Imagine Dragons,Birds,219440
3,2018-11-14 00:12,Imagine Dragons,Burn Out,273880
4,2018-11-14 00:16,Imagine Dragons,Real Life,247546
...,...,...,...,...
30822,2019-11-14 23:32,My Chemical Romance,Welcome to the Black Parade,311106
30823,2019-11-14 23:34,Tony Molina,Jasper's Theme,121509
30824,2019-11-14 23:38,Imagine Dragons,Cool Out,217866
30825,2019-11-14 23:41,5 Seconds of Summer,If Walls Could Talk,182292


In [8]:
artists = rawHist['artistName'].unique()
genreList = []
skippedList = []
painCount = 0
painOffset = 0
i = 0
while i < len(artists):
    try:
        print(i - painOffset)
        r = client.searchArtist(artists[i - painOffset])

        # Make sure the query returns something
        if len(r['artists']['items']) != 0:
            print('APPENDED GENRE')
            genreList.append(r['artists']['items'][0]['genres'])
        else:
            print('APPENDED NAN')
            genreList.append(np.nan)
            
        i += 1

        
    except SSLError:
        painCount += 1
        print("HELP HELP HELP HELP")
        
            
print(painCount)

0
https://api.spotify.com/v1/search?q=Imagine+Dragons&type=artist
200
APPENDED GENRE
1
https://api.spotify.com/v1/search?q=iLOVEFRiDAY&type=artist
200
APPENDED GENRE
2
https://api.spotify.com/v1/search?q=Panic%21+At+The+Disco&type=artist
200
APPENDED GENRE
3
https://api.spotify.com/v1/search?q=Oliver+Tree&type=artist
200
APPENDED GENRE
4
https://api.spotify.com/v1/search?q=88rising&type=artist
200
APPENDED GENRE
5
https://api.spotify.com/v1/search?q=bbno%24&type=artist
200
APPENDED GENRE
6
https://api.spotify.com/v1/search?q=atlas&type=artist
200
APPENDED GENRE
7
https://api.spotify.com/v1/search?q=Yung+Gravy&type=artist
200
APPENDED GENRE
8
https://api.spotify.com/v1/search?q=Joji&type=artist
200
APPENDED GENRE
9
https://api.spotify.com/v1/search?q=BROCKHAMPTON&type=artist
200
APPENDED GENRE
10
https://api.spotify.com/v1/search?q=Iamjakehill&type=artist
200
APPENDED GENRE
11
https://api.spotify.com/v1/search?q=Rich+Brian&type=artist
200
APPENDED GENRE
12
https://api.spotify.com/v1/sea

200
APPENDED GENRE
102
https://api.spotify.com/v1/search?q=baaskaT&type=artist
200
APPENDED GENRE
103
https://api.spotify.com/v1/search?q=Mura+Kami&type=artist
200
APPENDED GENRE
104
https://api.spotify.com/v1/search?q=Hue+Honey&type=artist
200
APPENDED GENRE
105
https://api.spotify.com/v1/search?q=%5Bbsd.u%5D&type=artist
200
APPENDED GENRE
106
https://api.spotify.com/v1/search?q=Chance+Thrash&type=artist
200
APPENDED GENRE
107
https://api.spotify.com/v1/search?q=Valntn&type=artist
200
APPENDED GENRE
108
https://api.spotify.com/v1/search?q=Squid+Ethics&type=artist
200
APPENDED GENRE
109
https://api.spotify.com/v1/search?q=The+1975&type=artist
200
APPENDED GENRE
110
https://api.spotify.com/v1/search?q=Fall+Out+Boy&type=artist
200
APPENDED GENRE
111
https://api.spotify.com/v1/search?q=Drake&type=artist
200
APPENDED GENRE
112
https://api.spotify.com/v1/search?q=Kesha&type=artist
200
APPENDED GENRE
113
https://api.spotify.com/v1/search?q=Viktor+Vaughn&type=artist
200
APPENDED GENRE
114
htt

200
APPENDED GENRE
200
https://api.spotify.com/v1/search?q=Kodak+Black&type=artist
200
APPENDED GENRE
201
https://api.spotify.com/v1/search?q=Bing+Crosby&type=artist
200
APPENDED GENRE
202
https://api.spotify.com/v1/search?q=Kawfee&type=artist
200
APPENDED GENRE
203
https://api.spotify.com/v1/search?q=Kyo+Itachi&type=artist
200
APPENDED GENRE
204
https://api.spotify.com/v1/search?q=BluntOne&type=artist
200
APPENDED GENRE
205
https://api.spotify.com/v1/search?q=Gus+Glasser&type=artist
200
APPENDED GENRE
206
https://api.spotify.com/v1/search?q=weird+inside&type=artist
200
APPENDED GENRE
207
https://api.spotify.com/v1/search?q=Flamingosis&type=artist
200
APPENDED GENRE
208
https://api.spotify.com/v1/search?q=B0nds&type=artist
200
APPENDED GENRE
209
https://api.spotify.com/v1/search?q=mt.+fujitive&type=artist
200
APPENDED GENRE
210
https://api.spotify.com/v1/search?q=Pete+Rock&type=artist
200
APPENDED GENRE
211
https://api.spotify.com/v1/search?q=Mujo&type=artist
200
APPENDED GENRE
212
htt

200
APPENDED GENRE
302
https://api.spotify.com/v1/search?q=Bodikhuu&type=artist
200
APPENDED GENRE
303
https://api.spotify.com/v1/search?q=Seneca+B&type=artist
200
APPENDED GENRE
304
https://api.spotify.com/v1/search?q=mommy&type=artist
200
APPENDED GENRE
305
https://api.spotify.com/v1/search?q=Mono%3AMassive&type=artist
200
APPENDED GENRE
306
https://api.spotify.com/v1/search?q=Jazzinuf&type=artist
200
APPENDED GENRE
307
https://api.spotify.com/v1/search?q=Cloudchord&type=artist
200
APPENDED GENRE
308
https://api.spotify.com/v1/search?q=Slumberville&type=artist
200
APPENDED GENRE
309
https://api.spotify.com/v1/search?q=Allmos&type=artist
200
APPENDED GENRE
310
https://api.spotify.com/v1/search?q=leaf+beach&type=artist
200
APPENDED GENRE
311
https://api.spotify.com/v1/search?q=Ian+Ewing&type=artist
200
APPENDED GENRE
312
https://api.spotify.com/v1/search?q=Dweeb&type=artist
200
APPENDED GENRE
313
https://api.spotify.com/v1/search?q=Dirty+Art+Club&type=artist
200
APPENDED GENRE
314
http

200
APPENDED GENRE
403
https://api.spotify.com/v1/search?q=Swit+Beats&type=artist
200
APPENDED GENRE
404
https://api.spotify.com/v1/search?q=Grim+Delarosa&type=artist
200
APPENDED GENRE
405
https://api.spotify.com/v1/search?q=Grant&type=artist
200
APPENDED GENRE
406
https://api.spotify.com/v1/search?q=Hunee&type=artist
200
APPENDED GENRE
407
https://api.spotify.com/v1/search?q=tricot&type=artist
200
APPENDED GENRE
408
https://api.spotify.com/v1/search?q=eery&type=artist
200
APPENDED GENRE
409
https://api.spotify.com/v1/search?q=Khalid&type=artist
200
APPENDED GENRE
410
https://api.spotify.com/v1/search?q=Strawberry+Girls&type=artist
200
APPENDED GENRE
411
https://api.spotify.com/v1/search?q=All+My+Alien+Sex+Friends&type=artist
200
APPENDED GENRE
412
https://api.spotify.com/v1/search?q=Earl+Sweatshirt&type=artist
200
APPENDED GENRE
413
https://api.spotify.com/v1/search?q=Rega&type=artist
200
APPENDED GENRE
414
https://api.spotify.com/v1/search?q=Daft+Punk&type=artist
200
APPENDED GENRE


...

APPENDED GENRE
1101
https://api.spotify.com/v1/search?q=Tomcbumpz&type=artist
200
APPENDED GENRE
1102
https://api.spotify.com/v1/search?q=3LLL&type=artist
200
APPENDED GENRE
1103
https://api.spotify.com/v1/search?q=Apollo+Brown&type=artist
200
APPENDED GENRE
1104
https://api.spotify.com/v1/search?q=All+Them+Witches&type=artist
200
APPENDED GENRE
1105
https://api.spotify.com/v1/search?q=G-Eazy&type=artist
200
APPENDED GENRE
1106
https://api.spotify.com/v1/search?q=Phoenix&type=artist
200
APPENDED GENRE
1107
https://api.spotify.com/v1/search?q=Neck+Deep&type=artist
200
APPENDED GENRE
1108
https://api.spotify.com/v1/search?q=Real+Friends&type=artist
200
APPENDED GENRE
1109
https://api.spotify.com/v1/search?q=Makeout&type=artist
200
APPENDED GENRE
1110
https://api.spotify.com/v1/search?q=The+Longshot&type=artist
200
APPENDED GENRE
1111
https://api.spotify.com/v1/search?q=Moose+Blood&type=artist
200
APPENDED GENRE
1112
https://api.spotify.com/v1/search?q=Falling+In+Reverse&type=artist
200
A

200
APPENDED GENRE
1200
https://api.spotify.com/v1/search?q=JOBA&type=artist
200
APPENDED GENRE
1201
https://api.spotify.com/v1/search?q=Ashley+O&type=artist
200
APPENDED GENRE
1202
https://api.spotify.com/v1/search?q=Blanco+Brown&type=artist
200
APPENDED GENRE
1203
https://api.spotify.com/v1/search?q=Mabel&type=artist
200
APPENDED GENRE
1204
https://api.spotify.com/v1/search?q=My+Brothers+And+I&type=artist
200
APPENDED GENRE
1205
https://api.spotify.com/v1/search?q=Social+House&type=artist
200
APPENDED GENRE
1206
https://api.spotify.com/v1/search?q=The+Head+and+the+Heart&type=artist
200
APPENDED GENRE
1207
https://api.spotify.com/v1/search?q=Aer&type=artist
200
APPENDED GENRE
1208
https://api.spotify.com/v1/search?q=Ansel+Elgort&type=artist
200
APPENDED GENRE
1209
https://api.spotify.com/v1/search?q=Holy+Mattress+Money&type=artist
200
APPENDED GENRE
1210
https://api.spotify.com/v1/search?q=Judah+%26+the+Lion&type=artist
200
APPENDED GENRE
1211
https://api.spotify.com/v1/search?q=LORYN

1297
https://api.spotify.com/v1/search?q=Cascade&type=artist
200
APPENDED GENRE
1298
https://api.spotify.com/v1/search?q=The+Polar+Boys&type=artist
200
APPENDED GENRE
1299
https://api.spotify.com/v1/search?q=Gaten+Matarazzo&type=artist
200
APPENDED GENRE
1300
https://api.spotify.com/v1/search?q=Sapphire&type=artist
200
APPENDED GENRE
1301
https://api.spotify.com/v1/search?q=Leisure+Club&type=artist
200
APPENDED GENRE
1302
https://api.spotify.com/v1/search?q=The+High+Curbs&type=artist
200
APPENDED GENRE
1303
https://api.spotify.com/v1/search?q=Manuel&type=artist
200
APPENDED GENRE
1304
https://api.spotify.com/v1/search?q=%C3%A3%E2%80%9A%C2%AA%C3%A3%C6%92%C2%AB%C3%A3%E2%80%9A%C2%B4%C3%A3%C6%92%C2%BC%C3%A3%C6%92%C2%AB+%C3%A3%C6%92%C5%B8%C3%A3%C6%92%E2%80%B0%C3%A3%C6%92%C2%AA&type=artist
200
APPENDED NAN
1305
https://api.spotify.com/v1/search?q=Vista+Kicks&type=artist
200
APPENDED GENRE
1306
https://api.spotify.com/v1/search?q=Hall+Johnson&type=artist
200
APPENDED GENRE
1307
https://api.sp

200
APPENDED GENRE
1393
https://api.spotify.com/v1/search?q=Tierra+Whack&type=artist
200
APPENDED GENRE
1394
https://api.spotify.com/v1/search?q=Kelela&type=artist
200
APPENDED GENRE
1395
https://api.spotify.com/v1/search?q=Jon+Hopkins&type=artist
200
APPENDED GENRE
1396
https://api.spotify.com/v1/search?q=Bazzi&type=artist
200
APPENDED GENRE
1397
https://api.spotify.com/v1/search?q=Yuno&type=artist
200
APPENDED GENRE
1398
https://api.spotify.com/v1/search?q=Yellow+Days&type=artist
200
APPENDED GENRE
1399
https://api.spotify.com/v1/search?q=Rynx&type=artist
200
APPENDED GENRE
1400
https://api.spotify.com/v1/search?q=keshi&type=artist
200
APPENDED GENRE
1401
https://api.spotify.com/v1/search?q=Kina&type=artist
200
APPENDED GENRE
1402
https://api.spotify.com/v1/search?q=Pia+Mia&type=artist
200
APPENDED GENRE
1403
https://api.spotify.com/v1/search?q=Vance+Joy&type=artist
200
APPENDED GENRE
1404
https://api.spotify.com/v1/search?q=Seinabo+Sey&type=artist
200
APPENDED GENRE
1405
https://api

200
APPENDED GENRE
1492
https://api.spotify.com/v1/search?q=Can%27t+Swim&type=artist
200
APPENDED GENRE
1493
https://api.spotify.com/v1/search?q=Speak+Low+If+You+Speak+Love&type=artist
200
APPENDED GENRE
1494
https://api.spotify.com/v1/search?q=Lydia&type=artist
200
APPENDED GENRE
1495
https://api.spotify.com/v1/search?q=Michael+Cera+Palin&type=artist
200
APPENDED GENRE
1496
https://api.spotify.com/v1/search?q=Citizen&type=artist
200
APPENDED GENRE
1497
https://api.spotify.com/v1/search?q=Issues&type=artist
200
APPENDED GENRE
1498
https://api.spotify.com/v1/search?q=Push+over&type=artist
200
APPENDED GENRE
1499
https://api.spotify.com/v1/search?q=From+First+To+Last&type=artist
200
APPENDED GENRE
1500
https://api.spotify.com/v1/search?q=I+The+Mighty&type=artist
200
APPENDED GENRE
1501
https://api.spotify.com/v1/search?q=Makari&type=artist
200
APPENDED GENRE
1502
https://api.spotify.com/v1/search?q=Polyenso&type=artist
200
APPENDED GENRE
1503
https://api.spotify.com/v1/search?q=Capstan&t

In [9]:
genreKey = dict(zip(artists, genreList))

In [10]:
genreKey['Zedd']

['complextro',
 'dance pop',
 'edm',
 'electro house',
 'electropop',
 'german techno',
 'pop',
 'post-teen pop',
 'tropical house']

In [11]:
dfGenres = pd.DataFrame({'artistName':artists, 'genres':genreList})

In [15]:
dfGenres.to_csv('dfGenres.csv', index=False)

In [17]:
rawHist.to_csv('totalHist.csv', index=False)

In [14]:
rawHist

,endTime,artistName,trackName,msPlayed
0,2018-11-14 00:01,Imagine Dragons,Stuck,190853
1,2018-11-14 00:04,Imagine Dragons,Love,166360
2,2018-11-14 00:08,Imagine Dragons,Birds,219440
3,2018-11-14 00:12,Imagine Dragons,Burn Out,273880
4,2018-11-14 00:16,Imagine Dragons,Real Life,247546
...,...,...,...,...
30822,2019-11-14 23:32,My Chemical Romance,Welcome to the Black Parade,311106
30823,2019-11-14 23:34,Tony Molina,Jasper's Theme,121509
30824,2019-11-14 23:38,Imagine Dragons,Cool Out,217866
30825,2019-11-14 23:41,5 Seconds of Summer,If Walls Could Talk,182292


In [15]:
def appendGenres(row):
    artist = row['artistName']
    return genreKey[artist]
    

In [22]:
genreCol = rawHist.apply(appendGenres, axis=1)
hist = rawHist.assign(genre = genreCol)


In [23]:
hist

,endTime,artistName,trackName,msPlayed,genre
0,2018-11-14 00:01,Imagine Dragons,Stuck,190853,"[modern rock, rock]"
1,2018-11-14 00:04,Imagine Dragons,Love,166360,"[modern rock, rock]"
2,2018-11-14 00:08,Imagine Dragons,Birds,219440,"[modern rock, rock]"
3,2018-11-14 00:12,Imagine Dragons,Burn Out,273880,"[modern rock, rock]"
4,2018-11-14 00:16,Imagine Dragons,Real Life,247546,"[modern rock, rock]"
...,...,...,...,...,...
30822,2019-11-14 23:32,My Chemical Romance,Welcome to the Black Parade,311106,"[emo, modern rock, pop punk]"
30823,2019-11-14 23:34,Tony Molina,Jasper's Theme,121509,"[indie punk, modern folk rock, modern power pop]"
30824,2019-11-14 23:38,Imagine Dragons,Cool Out,217866,"[modern rock, rock]"
30825,2019-11-14 23:41,5 Seconds of Summer,If Walls Could Talk,182292,"[boy band, dance pop, pop, post-teen pop]"


In [32]:
enc = OneHotEncoder()

In [89]:
hist = hist.dropna()
hist.isna().mean()

endTime       0.0
artistName    0.0
trackName     0.0
msPlayed      0.0
genre         0.0
dtype: float64

In [96]:
temp = []
for i in genreKey.values():
    if type(i) == list:
        for j in i:
            temp.append(j)

In [100]:
genres = pd.Series(temp).unique()

In [6]:
datetime.date.today()

NameError: name 'datetime' is not defined